# 4. Stitch outputs together
- Once you have fitted the weak, medium and dense diads, stitch them all together, and apply the Ne correction model

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import joblib
import DiadFit as pf
pf.__version__

'0.0.58'

In [2]:
import os
DayFolder=os.getcwd()
meta_path=DayFolder +  '\MetaData'
spectra_path=DayFolder + '\Spectra'

## Load in specra, and combine

In [3]:
import os.path
from os import path
if path.exists('Discarded_df.xlsx'):
    discard=pd.read_excel('Discarded_df.xlsx')
else:
    discard=None
if path.exists('Weak_Diads.xlsx'):
    grp1=pd.read_excel('Weak_Diads.xlsx')
else:
    grp1=None
if path.exists('Medium_Diads.xlsx'):
    grp2=pd.read_excel('Medium_Diads.xlsx')
else:
    grp2=None
if path.exists('Strong_Diads.xlsx'):
    grp3=pd.read_excel('Strong_Diads.xlsx')
else:
    grp3=None
df2=pd.concat([grp1, grp2, grp3], axis=0)
if discard is not None:
    discard_cols=discard[discard.columns.intersection(df2.columns)]
    df2=pd.concat([df2, discard_cols])

## First, get the metadata for this day

In [4]:
# Put the common string you can in all your Ne lines here
ID_str='_'
file_ext_meta='txt' # Will only take files of this type 
diad_meta=pf.get_files(path=meta_path,
file_ext='txt', exclude_str=['N', 'FG','Cap','window'],
            exclude_type='.png',   sort=True)
diad_meta

['POC1.txt',
 'POC10.txt',
 'POC11.txt',
 'POC12.txt',
 'POC13.txt',
 'POC14.txt',
 'POC15.txt',
 'POC16.txt',
 'POC17.txt',
 'POC18.txt',
 'POC19.txt',
 'POC2.txt',
 'POC20.txt',
 'POC21.txt',
 'POC22.txt',
 'POC23.txt',
 'POC24.txt',
 'POC25.txt',
 'POC26.txt',
 'POC27.txt',
 'POC28.txt',
 'POC29.txt',
 'POC3.txt',
 'POC30.txt',
 'POC31.txt',
 'POC32.txt',
 'POC33.txt',
 'POC34.txt',
 'POC35.txt',
 'POC36.txt',
 'POC37.txt',
 'POC38.txt',
 'POC39.txt',
 'POC4.txt',
 'POC40.txt',
 'POC41.txt',
 'POC42.txt',
 'POC43.txt',
 'POC44.txt',
 'POC45.txt',
 'POC46.txt',
 'POC47.txt',
 'POC48.txt',
 'POC49.txt',
 'POC5.txt',
 'POC50.txt',
 'POC51.txt',
 'POC52.txt',
 'POC53.txt',
 'POC54.txt',
 'POC55.txt',
 'POC56.txt',
 'POC57-0.04.txt',
 'POC58.txt',
 'POC6.txt',
 'POC7.txt',
 'POC8.txt',
 'POC9.txt']

## Now get all the important information out of the metadata files

In [5]:
meta=pf.stitch_metadata_in_loop_witec(path=meta_path, 
Allfiles=diad_meta, prefix=False,
trupower=False)
meta['filename'].iloc[0]

100%|██████████| 58/58 [00:03<00:00, 19.05it/s]

Done
Oof, try not to work after midnight!


'POC49.txt'

## Then get a simple file name you can stitch with the spectra

In [6]:
file_m=pf.extracting_filenames_generic(names=meta['filename'],
    prefix=False, str_prefix=" ",
   file_type='.txt')
# Checks they are all unique
file_m[0]

good job, no duplicate file names


'POC49'

In [7]:
type(file_m)

numpy.ndarray

## Now get filename from spectra into same form as metadata

In [8]:
type(df2['filename'].reset_index(drop=True))

pandas.core.series.Series

In [9]:
# Remove these to get the pure file name
file_s=pf.extracting_filenames_generic(
    prefix=False, str_prefix=" ",
    suffix=True, str_suffix='_CRR_DiadFit',
    names=df2['filename'].reset_index(drop=True),
   file_type='.txt')

file_s[0]

good job, no duplicate file names


'Cap19-05X2'

In [10]:
# Combining them together
meta['name_for_matching']=file_m
df2['name_for_matching']=file_s
df_combo=df2.merge(meta, on='name_for_matching')

In [11]:
from pickle import load
import pickle
import os
pickle_path=os.getcwd()
file_pkl=pickle_path + '/' + 'Ne_Line_Regression_Example_cali.pck'
with open(file_pkl, 'rb') as file_handle:
    my_loaded_model = pickle.load(file_handle)

In [12]:
df_combo['Ne_Correction']=my_loaded_model(df_combo['sec since midnight'])
df_combo.insert(1,'Corrected_Splitting', df_combo['Ne_Correction']*df_combo['Splitting'])

In [14]:
df_combo.to_excel('Calibration_fitting.xlsx')

In [15]:
df_combo

,Unnamed: 0,Corrected_Splitting,filename_x,Splitting,Split_err_abs,Split_err_quadrature,Diad1_Combofit_Cent,Diad1_cent_err,Diad1_Combofit_Height,Diad1_Voigt_Cent,...,Day,power (mW),Int_time (s),accumulations,Mag (X),duration,24hr_time,sec since midnight,Spectral Center,Ne_Correction
0,0,102.803073,POC51_CRR_DiadFit,102.740284,NaN,NaN,1286.697273,NaN,179.754189,1286.697273,...,29.0,NaN,60.0,3.0,5.0,"['0h', '3m', '2s']",12:17:28 AM\n,87448.0,1341.0,1.000611
1,0,102.797322,POC52_CRR_DiadFit,102.734429,NaN,NaN,1286.689648,NaN,244.745472,1286.689648,...,29.0,NaN,90.0,3.0,5.0,"['0h', '4m', '31s']",12:31:31 AM\n,88291.0,1341.0,1.000612
2,0,102.823765,POC53_CRR_DiadFit,102.760794,NaN,NaN,1286.675369,NaN,219.447379,1286.675369,...,29.0,NaN,90.0,3.0,5.0,"['0h', '4m', '32s']",12:39:13 AM\n,88753.0,1341.0,1.000613
3,0,102.822363,POC54_CRR_DiadFit,102.759330,NaN,NaN,1286.683795,NaN,183.617534,1286.683845,...,29.0,NaN,90.0,3.0,5.0,"['0h', '4m', '32s']",12:46:57 AM\n,89217.0,1341.0,1.000613
4,0,102.790489,POC55,102.727414,NaN,NaN,1286.707970,NaN,156.339980,1286.707970,...,29.0,NaN,90.0,3.0,5.0,"['0h', '4m', '32s']",12:54:26 AM\n,89666.0,1341.0,1.000614
5,0,102.792965,POC56_CRR_DiadFit,102.729825,NaN,NaN,1286.713940,NaN,131.045582,1286.713990,...,29.0,NaN,90.0,3.0,5.0,"['0h', '4m', '32s']",1:01:55 AM\n,90115.0,1341.0,1.000615
6,0,102.821918,POC57-0.04,102.758684,NaN,NaN,1286.687414,NaN,126.454443,1286.687414,...,29.0,NaN,90.0,5.0,5.0,"['0h', '7m', '32s']",1:10:45 AM\n,90645.0,1341.0,1.000615
7,0,102.791146,POC58_CRR_DiadFit,102.727841,NaN,NaN,1286.718188,NaN,118.641132,1286.718188,...,29.0,NaN,90.0,5.0,5.0,"['0h', '7m', '32s']",1:20:58 AM\n,91258.0,1341.0,1.000616
8,0,102.869746,POC39_CRR_DiadFit,102.807541,NaN,NaN,1286.514205,NaN,2008.914261,1286.514255,...,28.0,NaN,45.0,3.0,5.0,"['0h', '2m', '16s']",10:41:24 PM\n,81684.0,1341.0,1.000605
9,0,102.866306,POC40_CRR_DiadFit,102.804068,0.003790,0.002934,1286.536042,0.002740,1867.371873,1286.536092,...,28.0,NaN,45.0,3.0,5.0,"['0h', '2m', '16s']",10:47:37 PM\n,82057.0,1341.0,1.000605
